<a href="https://colab.research.google.com/github/giuliofortini/NLP_SQuAD_Project/blob/gpt/SQUAD_QGen_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Fine-tune a pretrained GPT and generate text! 🦾


In [2]:
#@title #**DASHBOARD**
MODEL_NAME = "GPT_SQUAD_answer-gen" #@param {type:"string"}

#@markdown ---
#@markdown ## Import settings:
GITHUB_REPO_NAME = "italian_horoscope_generator" #@param {type:"string"}
DATASET_NAME = "pulpfiction.txt" #@param {type:"string"}
DATASET_PATH = f"./{GITHUB_REPO_NAME}/datasets/{DATASET_NAME}"



#@markdown ---
#@markdown ## Model Training settings:
EPOCHS = "10" #@param [1, 2, 3, 5, 7, 10]
EPOCHS = int(EPOCHS)
PRETRAINED = "gpt2-medium" #@param ["LorenzoDeMattei/GePpeTto", "gpt2-medium"]
TEST_SIZE = 0.15 #@param {type:"slider", min:0, max:1, step:0.05}

#@markdown ---
#@markdown ## Generation settings:
PROMPTS = "JULES; VINCENT VEGA; JULES; VINCENT VEGA; JULES; VINCENT VEGA" #@param {type:"string"}
SEPARATOR = " ;" #@param {type:"string"}
PROMPTS = [f"[{p}]" for p in PROMPTS.split(SEPARATOR)]

#@markdown ---
#@markdown ## Export settings:
SAVE_MODEL_ON_DRIVE = True #@param {type:"boolean"}
OUT_HOROS_FILE = "generated.txt"
OUT_HOROS_FILE = f"{MODEL_NAME}_generated.txt"
DRIVE_PATH = f"/content/gdrive/My Drive/{MODEL_NAME}/"
DRIVE_MODEL_FOLDER = DRIVE_PATH + MODEL_NAME
MODELS_FOLDER = f"./{MODEL_NAME}_pretrained/"
MODEL_NAME = f"{MODEL_NAME}_{EPOCHS}"
MODEL_ARCHIVE_PATH = DRIVE_PATH+MODEL_NAME+".zip"


###################################à
print(f" Summary:\n{'-'*100}")
print(f" EPOCHS:                          {EPOCHS}")
print(f" GitHub repository name:          {GITHUB_REPO_NAME}")
print(f" Model Name:                      {MODEL_NAME}")
print(f" Local folder for saved models:   {MODELS_FOLDER}")
print(f" Drive folder for saved models:   {DRIVE_PATH}")
print(f" Model archive file name:         {MODEL_ARCHIVE_PATH}")
print(f" Dataset folder:                  {DATASET_PATH}")
print(f" Prompts for generation:          {PROMPTS}")


if SAVE_MODEL_ON_DRIVE:
  from google.colab import drive
  drive.mount('/content/gdrive')
  drive_path = DRIVE_PATH

 Summary:
----------------------------------------------------------------------------------------------------
 EPOCHS:                          10
 GitHub repository name:          italian_horoscope_generator
 Model Name:                      GPT_SQUAD_answer-gen_10
 Local folder for saved models:   ./GPT_SQUAD_answer-gen_pretrained/
 Drive folder for saved models:   /content/gdrive/My Drive/GPT_SQUAD_answer-gen/
 Model archive file name:         /content/gdrive/My Drive/GPT_SQUAD_answer-gen/GPT_SQUAD_answer-gen_10.zip
 Dataset folder:                  ./italian_horoscope_generator/datasets/pulpfiction.txt
 Prompts for generation:          ['[JULES; VINCENT VEGA; JULES; VINCENT VEGA; JULES; VINCENT VEGA]']
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Setup

In [ ]:
!pip install transformers
!nvidia-smi

from transformers import Trainer, TrainingArguments, AutoModelWithLMHead, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(PRETRAINED)
model = AutoModelWithLMHead.from_pretrained(PRETRAINED)


# clone git repository
import sys
!git clone "https://github.com/RiccardoCozzi96/italian_horoscope_generator"
sys.path.append(GITHUB_REPO_NAME+"/")


     |████████████████████████████████| 1.8MB 17.4MB/s 
     |████████████████████████████████| 2.9MB 49.9MB/s 
     |████████████████████████████████| 890kB 55.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=3b91c8e6b5c87f80bc5d5fb24d00074642943e3d5fc6201e466f87393cd13925
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Mon Feb  1 00:54:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|=====================

/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,



Cloning into 'italian_horoscope_generator'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 58 (delta 15), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (58/58), done.


## Dataset preparation



Dataset loading
>Note: the dataset is required to be in a simple .txt file in this form: 
> ```
[prompt 1] Sequence 1 ...
[prompt 2] Sequence 2 ...
...
```
*(with the prompt text between square brackets)*

In [ ]:
import json
from sklearn.model_selection import train_test_split
import pandas as pd

with open(DATASET_PATH) as f: 
  horoscopes = f.readlines()

print("Dataset sample: ")
for row in horoscopes[:5]:
  print(row)

Dataset sample: 
[RINGO] No, è troppo rischioso. Ho chiuso con queste stronzate.

[YOLANDA] Dici sempre così, ogni volta la stessa storia: "Ho chiuso, mai più, è troppo pericoloso...".

[RINGO] Lo so che lo dico sempre, ho anche ragione.

[YOLANDA] Ma tendi a dimenticartene dopo un giorno o due.

[RINGO] Ma i giorni in cui dimentico sono finiti. Stanno per cominciare i giorni in cui ricordo.



Train / Test split

In [ ]:
train, test = train_test_split(horoscopes, test_size=TEST_SIZE)
train_path = "train.txt"
test_path = "test.txt"

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))
print("\nTrain sample:\n", train[0])
print("\nTest sample:\n", test[0])

with open(train_path, "w") as train_file:
  train_file.writelines(train)
with open(test_path, "w") as test_file:
  test_file.writelines(test)

Train dataset length: 1138
Test dataset length: 201

Train sample:
 [BUTCH] Beh, dovresti essere contenta, perché ce l'hai.


Test sample:
 [JULES] Porca troia, è gelata!



Creating TextDataset

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset, test_dataset ,data_collator = load_dataset(train_path, test_path, tokenizer)

/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


## Initialize `Trainer` with `TrainingArguments` and GPT-2 model

The [Trainer](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Trainer) class provides an API for feature-complete training. It is used in most of the [example scripts](https://huggingface.co/transformers/examples.html) from Huggingface. Before we can instantiate our `Trainer` we need to download our GPT-2 model and create a [TrainingArguments](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments) to access all the points of customization during training. In the `TrainingArguments`, we can define the Hyperparameters we are going to use in the training process like our `learning_rate`, `num_train_epochs`, or  `per_device_train_batch_size`. A complete list can you find [here](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments).

In [ ]:

training_args = TrainingArguments(
    output_dir = MODEL_NAME, # The output directory
    overwrite_output_dir=True, # overwrite the content of the output directory
    num_train_epochs = EPOCHS, # number of training epochs
    per_device_train_batch_size = 32, # batch size for training
    per_device_eval_batch_size = 64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps = 800, # after # steps model is saved 
    warmup_steps = 500,# number of warmup steps for learning rate scheduler
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    #prediction_loss_only=True,
)

## Training

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=70, training_loss=3.8287471226283483, metrics={'train_runtime': 67.0344, 'train_samples_per_second': 1.044, 'total_flos': 183967757107200, 'epoch': 10.0})

##Saving the model

After training is done you can save the model by calling `save_model()`. This will save the trained model to our `output_dir` from our `TrainingArguments`.

In [ ]:
trainer.save_model()

In [ ]:
import os
import shutil

# zip the model folder
print("Creating archive...", end=" ")
shutil.make_archive(MODEL_NAME, 'zip', MODEL_NAME)
print("done: ", MODEL_NAME+".zip")

# save on Google Drive
if SAVE_MODEL_ON_DRIVE:
  
  if not os.path.exists(drive_path):
    os.makedirs(drive_path)
    print("path created on drive: ", drive_path)
  else:
    print("path already existing: ", drive_path)
  
  print("Saving archive on drive...", end="")
  shutil.copy(f"./{MODEL_NAME}.zip", f"{drive_path}{MODEL_NAME}.zip")
  print("done.")

Creating archive... done:  GPTMovie_10.zip
path already existing:  /content/gdrive/My Drive/GPTMovie/
Saving archive on drive...done.


## Text Generation



To test the model we are going to use another [highlight of the transformers library](https://huggingface.co/transformers/main_classes/pipelines.html?highlight=pipelines) called `pipeline`. [Pipelines](https://huggingface.co/transformers/main_classes/pipelines.html?highlight=pipelines) are objects that offer a simple API dedicated to several tasks, among others also `text-generation`

In [ ]:
def clean(text):
  a = text[:text.find("]")+1]
  b = text[text.find("]"):]
  c = b[:b.find("[")].replace("]", "")
  c = c.replace("\n", " ").strip()
  return (a[1:-1], c)

def format(sign_body):
  sign, body = sign_body
  return f"\n[{sign}]\n{body}\n"

In [ ]:
from transformers import pipeline
text_generator = pipeline('text-generation', model=MODEL_NAME, tokenizer=tokenizer, config={'max_length':800})

In [ ]:
samples_outputs = text_generator(
    PROMPTS,
    do_sample=True,
    max_length=500,
    top_k=50,
    top_p=0.95,
    num_return_sequences=1
)

In [ ]:
# save cleaned texts
outputs = []
for prompt, sample in zip(PROMPTS, samples_outputs):
  text = clean(sample[0]["generated_text"].replace("\n", " "))
  line = f"{prompt} {text[1]}"
  outputs.append(line)

[JULES;] Ma di che hai paura di questa mia stupidità?
[VINCENT] Non voglio che questo paese diventi troppo "retta da dio"!
[VEGA;] Tu hai portato un'altra volta in America? Ti sei stufato? Non ti sembra di no!
[JULES;] Buffy e l'inferno. WILLIAMS
[VINCENT] Se hai in mente di farti un pensierino, lo fai solo tu! Lo fanno per una settimana.
[VEGA;] Ma chi vi porterà a casa?
[JULES;] Non voglio fare del male, ma non voglio farti del male.
[VINCENT] Sì, sono qui dentro."
[VEGA] Azz..ma non è un cazzo che pensi...pure questo sarebbe un buon libro.  Links http://www.ilgazzettino.it/main.php3?Luogo=Rovigo&Data=2005-7-30&Pagina=SANGO Le nuove regole di sicurezza per i siti di scambio e lo scambio di merci tra Stati Uniti e Italia: un nuovo modo per combattere le crisi dei missili a testata nucleare. L'Enea, infatti, lancia l'allarme nucleare e si è detto sorpreso che le industrie statunitensi hanno chiuso tutte le porte della catena di navigazione. Eni a San Pietroburgo, e, ma, alla fine dell'

## View results

In [ ]:
for output in outputs:
  print("> ", output)

>  [JULES;] Ma di che hai paura di questa mia stupidità?
>  [VINCENT] Non voglio che questo paese diventi troppo "retta da dio"!
>  [VEGA;] Tu hai portato un'altra volta in America? Ti sei stufato? Non ti sembra di no!
>  [JULES;] Buffy e l'inferno. WILLIAMS
>  [VINCENT] Se hai in mente di farti un pensierino, lo fai solo tu! Lo fanno per una settimana.
>  [VEGA;] Ma chi vi porterà a casa?
>  [JULES;] Non voglio fare del male, ma non voglio farti del male.
>  [VINCENT] Sì, sono qui dentro."
>  [VEGA] Azz..ma non è un cazzo che pensi...pure questo sarebbe un buon libro.  Links http://www.ilgazzettino.it/main.php3?Luogo=Rovigo&Data=2005-7-30&Pagina=SANGO Le nuove regole di sicurezza per i siti di scambio e lo scambio di merci tra Stati Uniti e Italia: un nuovo modo per combattere le crisi dei missili a testata nucleare. L'Enea, infatti, lancia l'allarme nucleare e si è detto sorpreso che le industrie statunitensi hanno chiuso tutte le porte della catena di navigazione. Eni a San Pietrobu

## Saving outputs

In [ ]:
with open(OUT_HOROS_FILE, "w") as out:
  out.writelines(outputs)

shutil.copy(OUT_HOROS_FILE, "/content/gdrive/My Drive/GPTFox")

'/content/gdrive/My Drive/GPTFox/GPTMovie_generated.txt'

## Evaluation

In [ ]:
sample = samples_outputs[0][0]["generated_text"][:240]
print("SAMPLE: ", sample)

with open("train.txt") as f:
  GT = f.read()


def find_substr(string, text, print_window=False, window_size=30):
  if string in text:
    position = text.find(string)
    if print_window: 
      window = text[ position-window_size : position+len(string)+window_size ]
      window = window.replace(string, string.upper())
      print(window)
    return True
  else:
    return False


def get_all_ngrams(string, n):
  ngrams = []
  string_len = len(string.split(" "))
  for i in range(0, string_len-n+1, 1):
    ngram = " ".join(string.split(" ")[i:i+n])
    ngrams.append(ngram.strip())
  return ngrams


def count_ngram_plagiarism(string, text, n_values, verbose=False):
  ngram_plagiarism = { n:0 for n in n_values }
  for n in n_values[::-1]: # read backward
    for i, ngram in enumerate(get_all_ngrams(string, n)):
      found = find_substr(ngram, text)
      if verbose: print(f"[{i}-{i+n}]\t{found}\t{ngram}")
      if found: ngram_plagiarism[n] += 1
    

  return ngram_plagiarism


def plagiarism_score(plagiarism):
  positive_plag = [n for n in list(plagiarism.keys()) if plagiarism[n] != 0]
  if positive_plag == []: 
    return 0

  n_max = max(positive_plag)
  score = 1 / (n_max * plagiarism[n_max])
  return round(score, 3)


#find_substr("qualche difficoltà a rapportarvi con", ground_truth, print_window=True)
plag_test = count_ngram_plagiarism(sample, GT, [5,6,7,8,9])
print(plag_test, plagiarism_score(plag_test))

SAMPLE:  [JULES;] Ma di che hai paura di questa mia stupidità?
[MIA] Mi chiamo Flora!
[JULES] Ma di che sei fiero?
[BUTCH] Ah, no, ma come, Flora!
[VINCENT] Oh, sì, ma non sono il figlio del fratello.
[JULES] Come sarebbe diventato grande il tuo mat
{5: 0, 6: 0, 7: 0, 8: 0, 9: 0} 0


In [ ]:
for oroscopo in outputs:
  plagi = count_ngram_plagiarism(oroscopo, GT, [5, 6, 7, 8, 9, 10], verbose=False)
  print("\n", plagiarism_score(plagi), "\t", oroscopo)


 0 	 [JULES;] Ma di che hai paura di questa mia stupidità?

 0 	 [VINCENT] Non voglio che questo paese diventi troppo "retta da dio"!

 0 	 [VEGA;] Tu hai portato un'altra volta in America? Ti sei stufato? Non ti sembra di no!

 0 	 [JULES;] Buffy e l'inferno. WILLIAMS

 0 	 [VINCENT] Se hai in mente di farti un pensierino, lo fai solo tu! Lo fanno per una settimana.

 0 	 [VEGA;] Ma chi vi porterà a casa?

 0 	 [JULES;] Non voglio fare del male, ma non voglio farti del male.

 0 	 [VINCENT] Sì, sono qui dentro."

 0.008 	 [VEGA] Azz..ma non è un cazzo che pensi...pure questo sarebbe un buon libro.  Links http://www.ilgazzettino.it/main.php3?Luogo=Rovigo&Data=2005-7-30&Pagina=SANGO Le nuove regole di sicurezza per i siti di scambio e lo scambio di merci tra Stati Uniti e Italia: un nuovo modo per combattere le crisi dei missili a testata nucleare. L'Enea, infatti, lancia l'allarme nucleare e si è detto sorpreso che le industrie statunitensi hanno chiuso tutte le porte della catena di 

In [ ]:
import time
print("Terminated at", str(time.ctime()))

Terminated at Mon Feb  1 01:01:29 2021
